# Simulate Women's Tourney

Generated brackets based on model probabilities and public picks. 

In [1]:
season = 2024
playin_losers = (  # remove play-in losers from seeding data
    3357,  # Sacred Heart
    3162,  # Columbia
    3120,  # Auburn
    3221,  # Holy Cross
)

season

2024

In [2]:
import pandas as pd

pd.set_option('display.max_columns', 100)

df_seeds = pd.read_csv(r'..\data\unprocessed\kaggle\WNCAATourneySeeds.csv')

df_seeds = df_seeds.loc[df_seeds['Season'] == season, :].reset_index(drop=True)

df_seeds.insert(2, 'Play In', df_seeds['Seed'].str.endswith(('a', 'b')))
df_seeds.insert(2, 'Region', df_seeds['Seed'].str[0])
df_seeds['Seed'] = df_seeds['Seed'].str.extract('(\d+)').astype(int)

df_seeds = df_seeds.loc[~df_seeds['TeamID'].isin(playin_losers), :].reset_index(drop=True)

df_seeds.insert(1, 'Region Seed', df_seeds['Region'] + df_seeds['Seed'].astype(str).str.zfill(2))

df_seeds

,Season,Region Seed,Seed,Region,Play In,TeamID
0,2024,W01,1,W,False,3376
1,2024,W02,2,W,False,3323
2,2024,W03,3,W,False,3333
3,2024,W04,4,W,False,3231
4,2024,W05,5,W,False,3328
...,...,...,...,...,...,...
59,2024,Z12,12,Z,True,3435
60,2024,Z13,13,Z,False,3267
61,2024,Z14,14,Z,False,3238
62,2024,Z15,15,Z,False,3263


### Simulate Tournament

I think it might make more sense to just go through all 63 matchups as a dictionary, adding future rounds as you go

In [3]:
df_matrix = pd.read_csv(
    fr'..\data\preprocessed\womens_year_data\{season}_womens_matchup_matrix.csv',
    index_col='Team A ID',
).rename_axis(index=None)

df_matrix.columns = df_matrix.columns.astype(int)

df_matrix

,3104,3112,3124,3151,3160,3163,3166,3179,3180,3181,3186,3193,3195,3199,3211,3231,3234,3235,3238,3242,3243,3245,3257,3261,3263,3266,3267,3268,3276,3277,3279,3292,3301,3304,3313,3314,3323,3326,3328,3333,3339,3342,3343,3349,3350,3355,3376,3390,3393,3394,3397,3400,3401,3404,3414,3417,3424,3425,3428,3435,3439,3452,3453,3465
3104,NaN,0.521184,0.347804,0.845126,0.250675,0.101080,0.380383,0.699852,0.925676,0.404095,0.832038,0.768824,0.630972,0.453428,0.256026,0.240352,0.114150,0.437386,0.824498,0.517816,0.281354,0.862754,0.371704,0.150066,0.909535,0.544749,0.847410,0.361256,0.574312,0.468190,0.415579,0.654988,0.205879,0.482425,0.909899,0.477075,0.173332,0.203785,0.478682,0.292676,0.868428,0.971064,0.474392,0.898960,0.638667,0.660787,0.024604,0.146368,0.438078,0.948260,0.469361,0.171866,0.647185,0.953701,0.903293,0.163650,0.598642,0.208285,0.267357,0.660744,0.307658,0.427203,0.717745,0.904074
3112,0.477344,NaN,0.331136,0.840633,0.297390,0.098005,0.423056,0.748483,0.943858,0.394052,0.836875,0.797877,0.695348,0.524942,0.297484,0.286420,0.155751,0.492103,0.823959,0.548234,0.287015,0.912708,0.322086,0.172629,0.919414,0.598075,0.867872,0.471737,0.567238,0.494023,0.436722,0.622254,0.259438,0.447645,0.880990,0.474341,0.190819,0.213300,0.408628,0.247886,0.889932,0.966642,0.498033,0.903418,0.680303,0.632908,0.037568,0.179224,0.470503,0.949630,0.405006,0.167244,0.624448,0.954706,0.910215,0.147374,0.559064,0.156113,0.406660,0.681619,0.274489,0.390448,0.746872,0.874859
3124,0.652978,0.659506,NaN,0.929609,0.442287,0.192494,0.605474,0.802819,0.968277,0.645152,0.918279,0.848649,0.758043,0.632431,0.496476,0.393174,0.189289,0.561117,0.912979,0.706138,0.443938,0.940537,0.478953,0.309574,0.960341,0.732502,0.903784,0.553748,0.712977,0.658833,0.700559,0.768428,0.400755,0.625928,0.940427,0.631152,0.335975,0.350932,0.550915,0.412693,0.932795,0.976943,0.672441,0.944655,0.811523,0.776252,0.060435,0.262238,0.623866,0.967238,0.573242,0.316506,0.777282,0.966158,0.957429,0.245482,0.741917,0.316162,0.437016,0.740714,0.439934,0.676922,0.831170,0.938884
3151,0.150750,0.155360,0.067746,NaN,0.061217,0.030585,0.093395,0.207331,0.691655,0.093231,0.349581,0.285181,0.224864,0.105663,0.065868,0.044719,0.028496,0.176310,0.469457,0.165594,0.065438,0.619395,0.090137,0.034779,0.589135,0.162391,0.347589,0.098561,0.178657,0.113730,0.128612,0.186929,0.049727,0.131426,0.666136,0.127699,0.044669,0.042566,0.088467,0.076042,0.417571,0.939272,0.116811,0.563212,0.169017,0.245131,0.018210,0.029026,0.147821,0.774531,0.107235,0.035490,0.182853,0.820359,0.664687,0.035136,0.221535,0.053285,0.055186,0.159928,0.059495,0.098371,0.256384,0.464767
3160,0.752506,0.701303,0.558338,0.937877,NaN,0.276365,0.706403,0.834906,0.964217,0.680658,0.928875,0.887042,0.818318,0.720530,0.543535,0.494662,0.302870,0.679713,0.940728,0.677175,0.553016,0.951482,0.535063,0.477424,0.955726,0.742203,0.913576,0.613433,0.751757,0.662192,0.725364,0.781370,0.376043,0.697753,0.955758,0.669940,0.431768,0.388586,0.666719,0.496586,0.930709,0.973464,0.709527,0.960398,0.846444,0.817831,0.082131,0.279669,0.653113,0.969401,0.667435,0.284810,0.824719,0.968738,0.954623,0.258008,0.787519,0.460749,0.517382,0.830674,0.485833,0.660561,0.849071,0.946706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3435,0.339104,0.319329,0.255686,0.833759,0.168091,0.054151,0.301576,0.511483,0.927389,0.270354,0.775962,0.641790,0.473161,0.320910,0.171472,0.184116,0.071823,0.378934,0.756970,0.425249,0.257226,0.827675,0.232086,0.112074,0.845883,0.409321,0.767428,0.260729,0.367173,0.354177,0.276671,0.467289,0.123574,0.299480,0.859290,0.322145,0.118175,0.108296,0.307109,0.149732,0.808941,0.959817,0.317879,0.838821,0.455278,0.433231,0.019508,0.072967,0.348004,0.921435,0.234085,0.071139,0.479711,0.957660,0.849914,0.059182,0.486037,0.119486,0.173977,NaN

In [4]:
id_to_region_seed = dict(zip(df_seeds['TeamID'], df_seeds['Region Seed']))

df_matrix.index = df_matrix.index.map(id_to_region_seed)
df_matrix.columns = df_matrix.columns.map(id_to_region_seed)

df_matrix

,X08,Z11,Z05,X14,Y05,Z03,Y07,Y12,X16,Z07,W14,W13,W12,X09,X04,W04,Y01,X07,Z14,Z08,Y04,W15,Y06,Y03,Z15,W10,Z13,X10,Z09,W09,W07,Y11,X03,W06,X15,W08,W02,Z02,W05,W03,Y13,W16,Y09,Y14,Z10,X12,W01,X02,Z06,Z16,X06,X01,W11,Y16,X13,Y02,Y10,Z01,X05,Z12,Z04,Y08,X11,Y15
X08,NaN,0.521184,0.347804,0.845126,0.250675,0.101080,0.380383,0.699852,0.925676,0.404095,0.832038,0.768824,0.630972,0.453428,0.256026,0.240352,0.114150,0.437386,0.824498,0.517816,0.281354,0.862754,0.371704,0.150066,0.909535,0.544749,0.847410,0.361256,0.574312,0.468190,0.415579,0.654988,0.205879,0.482425,0.909899,0.477075,0.173332,0.203785,0.478682,0.292676,0.868428,0.971064,0.474392,0.898960,0.638667,0.660787,0.024604,0.146368,0.438078,0.948260,0.469361,0.171866,0.647185,0.953701,0.903293,0.163650,0.598642,0.208285,0.267357,0.660744,0.307658,0.427203,0.717745,0.904074
Z11,0.477344,NaN,0.331136,0.840633,0.297390,0.098005,0.423056,0.748483,0.943858,0.394052,0.836875,0.797877,0.695348,0.524942,0.297484,0.286420,0.155751,0.492103,0.823959,0.548234,0.287015,0.912708,0.322086,0.172629,0.919414,0.598075,0.867872,0.471737,0.567238,0.494023,0.436722,0.622254,0.259438,0.447645,0.880990,0.474341,0.190819,0.213300,0.408628,0.247886,0.889932,0.966642,0.498033,0.903418,0.680303,0.632908,0.037568,0.179224,0.470503,0.949630,0.405006,0.167244,0.624448,0.954706,0.910215,0.147374,0.559064,0.156113,0.406660,0.681619,0.274489,0.390448,0.746872,0.874859
Z05,0.652978,0.659506,NaN,0.929609,0.442287,0.192494,0.605474,0.802819,0.968277,0.645152,0.918279,0.848649,0.758043,0.632431,0.496476,0.393174,0.189289,0.561117,0.912979,0.706138,0.443938,0.940537,0.478953,0.309574,0.960341,0.732502,0.903784,0.553748,0.712977,0.658833,0.700559,0.768428,0.400755,0.625928,0.940427,0.631152,0.335975,0.350932,0.550915,0.412693,0.932795,0.976943,0.672441,0.944655,0.811523,0.776252,0.060435,0.262238,0.623866,0.967238,0.573242,0.316506,0.777282,0.966158,0.957429,0.245482,0.741917,0.316162,0.437016,0.740714,0.439934,0.676922,0.831170,0.938884
X14,0.150750,0.155360,0.067746,NaN,0.061217,0.030585,0.093395,0.207331,0.691655,0.093231,0.349581,0.285181,0.224864,0.105663,0.065868,0.044719,0.028496,0.176310,0.469457,0.165594,0.065438,0.619395,0.090137,0.034779,0.589135,0.162391,0.347589,0.098561,0.178657,0.113730,0.128612,0.186929,0.049727,0.131426,0.666136,0.127699,0.044669,0.042566,0.088467,0.076042,0.417571,0.939272,0.116811,0.563212,0.169017,0.245131,0.018210,0.029026,0.147821,0.774531,0.107235,0.035490,0.182853,0.820359,0.664687,0.035136,0.221535,0.053285,0.055186,0.159928,0.059495,0.098371,0.256384,0.464767
Y05,0.752506,0.701303,0.558338,0.937877,NaN,0.276365,0.706403,0.834906,0.964217,0.680658,0.928875,0.887042,0.818318,0.720530,0.543535,0.494662,0.302870,0.679713,0.940728,0.677175,0.553016,0.951482,0.535063,0.477424,0.955726,0.742203,0.913576,0.613433,0.751757,0.662192,0.725364,0.781370,0.376043,0.697753,0.955758,0.669940,0.431768,0.388586,0.666719,0.496586,0.930709,0.973464,0.709527,0.960398,0.846444,0.817831,0.082131,0.279669,0.653113,0.969401,0.667435,0.284810,0.824719,0.968738,0.954623,0.258008,0.787519,0.460749,0.517382,0.830674,0.485833,0.660561,0.849071,0.946706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z12,0.339104,0.319329,0.255686,0.833759,0.168091,0.054151,0.301576,0.511483,0.927389,0.270354,0.775962,0.641790,0.473161,0.320910,0.171472,0.184116,0.071823,0.378934,0.756970,0.425249,0.257226,0.827675,0.232086,0.112074,0.845883,0.409321,0.767428,0.260729,0.367173,0.354177,0.276671,0.467289,0.123574,0.299480,0.859290,0.322145,0.118175,0.108296,0.307109,0.149732,0.808941,0.959817,0.317879,0.838821,0.455278,0.433231,0.019508,0.072967,0.348004,0.921435,0.234085,0.071139,0.479711,0.957660,0.849914,0.059182,0.486037,0.119486,0.173977,NaN,0.214202,0.286497,0.592927,0.842414
Z04,0.692128,0.728256,0.560401,0.

In [5]:
df_matrix_opposite = 1 - df_matrix  # prob of team losing

df_matrix_opposite

,X08,Z11,Z05,X14,Y05,Z03,Y07,Y12,X16,Z07,W14,W13,W12,X09,X04,W04,Y01,X07,Z14,Z08,Y04,W15,Y06,Y03,Z15,W10,Z13,X10,Z09,W09,W07,Y11,X03,W06,X15,W08,W02,Z02,W05,W03,Y13,W16,Y09,Y14,Z10,X12,W01,X02,Z06,Z16,X06,X01,W11,Y16,X13,Y02,Y10,Z01,X05,Z12,Z04,Y08,X11,Y15
X08,NaN,0.478816,0.652196,0.154874,0.749325,0.898920,0.619617,0.300148,0.074324,0.595905,0.167962,0.231176,0.369028,0.546572,0.743974,0.759648,0.885850,0.562614,0.175502,0.482184,0.718646,0.137246,0.628296,0.849934,0.090465,0.455251,0.152590,0.638744,0.425688,0.531810,0.584421,0.345012,0.794121,0.517575,0.090101,0.522925,0.826668,0.796215,0.521318,0.707324,0.131572,0.028936,0.525608,0.101040,0.361333,0.339213,0.975396,0.853632,0.561922,0.051740,0.530639,0.828134,0.352815,0.046299,0.096707,0.836350,0.401358,0.791716,0.732643,0.339256,0.692342,0.572797,0.282255,0.095926
Z11,0.522656,NaN,0.668864,0.159367,0.702610,0.901995,0.576944,0.251517,0.056142,0.605948,0.163125,0.202123,0.304652,0.475058,0.702516,0.713580,0.844249,0.507897,0.176041,0.451766,0.712985,0.087292,0.677914,0.827371,0.080586,0.401925,0.132128,0.528263,0.432762,0.505977,0.563278,0.377746,0.740562,0.552355,0.119010,0.525659,0.809181,0.786700,0.591372,0.752114,0.110068,0.033358,0.501967,0.096582,0.319697,0.367092,0.962432,0.820776,0.529497,0.050370,0.594994,0.832756,0.375552,0.045294,0.089785,0.852626,0.440936,0.843887,0.593340,0.318381,0.725511,0.609552,0.253128,0.125141
Z05,0.347022,0.340494,NaN,0.070391,0.557713,0.807506,0.394526,0.197181,0.031723,0.354848,0.081721,0.151351,0.241957,0.367569,0.503524,0.606826,0.810711,0.438883,0.087021,0.293862,0.556062,0.059463,0.521047,0.690426,0.039659,0.267498,0.096216,0.446252,0.287023,0.341167,0.299441,0.231572,0.599245,0.374072,0.059573,0.368848,0.664025,0.649068,0.449085,0.587306,0.067205,0.023057,0.327559,0.055345,0.188477,0.223748,0.939565,0.737762,0.376134,0.032762,0.426758,0.683494,0.222718,0.033842,0.042571,0.754518,0.258083,0.683838,0.562984,0.259286,0.560066,0.323078,0.168830,0.061116
X14,0.849250,0.844640,0.932254,NaN,0.938783,0.969415,0.906605,0.792669,0.308345,0.906769,0.650419,0.714819,0.775136,0.894337,0.934132,0.955281,0.971504,0.823690,0.530543,0.834406,0.934562,0.380605,0.909863,0.965221,0.410865,0.837609,0.652411,0.901439,0.821343,0.886270,0.871388,0.813071,0.950273,0.868574,0.333864,0.872301,0.955331,0.957434,0.911533,0.923958,0.582429,0.060728,0.883189,0.436788,0.830983,0.754869,0.981790,0.970974,0.852179,0.225469,0.892765,0.964510,0.817147,0.179641,0.335313,0.964864,0.778465,0.946715,0.944814,0.840072,0.940505,0.901629,0.743616,0.535233
Y05,0.247494,0.298697,0.441662,0.062123,NaN,0.723635,0.293597,0.165094,0.035783,0.319342,0.071125,0.112958,0.181682,0.279470,0.456465,0.505338,0.697130,0.320287,0.059272,0.322825,0.446984,0.048518,0.464937,0.522576,0.044274,0.257797,0.086424,0.386567,0.248243,0.337808,0.274636,0.218630,0.623957,0.302247,0.044242,0.330060,0.568232,0.611414,0.333281,0.503414,0.069291,0.026536,0.290473,0.039602,0.153556,0.182169,0.917869,0.720331,0.346887,0.030599,0.332565,0.715190,0.175281,0.031262,0.045377,0.741992,0.212481,0.539251,0.482618,0.169326,0.514167,0.339439,0.150929,0.053294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z12,0.660896,0.680671,0.744314,0.166241,0.831909,0.945849,0.698424,0.488517,0.072611,0.729646,0.224038,0.358210,0.526839,0.679090,0.828528,0.815884,0.928177,0.621066,0.243030,0.574751,0.742774,0.172325,0.767914,0.887926,0.154117,0.590679,0.232572,0.739271,0.632827,0.645823,0.723329,0.532711,0.876426,0.700520,0.140710,0.677855,0.881825,0.891704,0.692891,0.850268,0.191059,0.040183,0.682121,0.161179,0.544722,0.566769,0.980492,0.927033,0.651996,0.078565,0.765915,0.928861,0.520289,0.042340,0.150086,0.940818,0.513963,0.880514,0.826023,NaN,0.785798,0.713503,0.407073,0.157586
Z04,0.307872,0.271744,0.439599,0.

In [6]:
import numpy as np

def simulate_tournament() -> dict:
    """
    Simulate the tournament in Kaggle submission format
    """
    # ROUND 1
    round_1 = {
        f'R1{region}{i}': np.random.choice(
            (f'{region}{i:02d}', f'{region}{17-i:02d}'), 
            p=(
                df_matrix.loc[f'{region}{i:02d}', f'{region}{17-i:02d}'], 
                df_matrix_opposite.loc[f'{region}{i:02d}', f'{region}{17-i:02d}']
            )
        )
        for region in ('W', 'X', 'Y', 'Z')
        for i in (1, 8, 5, 4, 6, 3, 7, 2)
    }

    # ROUND 2
    round_2 = {
        f'R2{region}{i}': np.random.choice(
            (round_1[f'R1{region}{i:01d}'], round_1[f'R1{region}{9-i:01d}']), 
            p=(
                df_matrix.loc[round_1[f'R1{region}{i:01d}'], round_1[f'R1{region}{9-i:01d}']], 
                df_matrix_opposite.loc[round_1[f'R1{region}{i:01d}'], round_1[f'R1{region}{9-i:01d}']]
            )
        )
        for region in ('W', 'X', 'Y', 'Z')
        for i in (1, 4, 3, 2)
    }

    # ROUND 3
    round_3 = {
        f'R3{region}{i}': np.random.choice(
            (round_2[f'R2{region}{i:01d}'], round_2[f'R2{region}{5-i:01d}']), 
            p=(
                df_matrix.loc[round_2[f'R2{region}{i:01d}'], round_2[f'R2{region}{5-i:01d}']], 
                df_matrix_opposite.loc[round_2[f'R2{region}{i:01d}'], round_2[f'R2{region}{5-i:01d}']]
            )
        )
        for region in ('W', 'X', 'Y', 'Z')
        for i in (1, 2)
    }

    # ROUND 4
    round_4 = {
        f'R4{region}{i}': np.random.choice(
            (round_3[f'R3{region}{i:01d}'], round_3[f'R3{region}{3-i:01d}']), 
            p=(
                df_matrix.loc[round_3[f'R3{region}{i:01d}'], round_3[f'R3{region}{3-i:01d}']], 
                df_matrix_opposite.loc[round_3[f'R3{region}{i:01d}'], round_3[f'R3{region}{3-i:01d}']]
            )
        )
        for region in ('W', 'X', 'Y', 'Z')
        for i in (1, )
    }

    # ROUND 5
    round_5 = {
        'R5WX': np.random.choice(
            (round_4['R4W1'], round_4['R4X1']),
            p=(
                df_matrix.loc[round_4['R4W1'], round_4['R4X1']],
                df_matrix_opposite.loc[round_4['R4W1'], round_4['R4X1']],
            )
        ),
        'R5YZ': np.random.choice(
            (round_4['R4Y1'], round_4['R4Z1']),
            p=(
                df_matrix.loc[round_4['R4Y1'], round_4['R4Z1']],
                df_matrix_opposite.loc[round_4['R4Y1'], round_4['R4Z1']],
            )
        ),
    }

    # ROUND 6
    round_6 = {
        'R6CH': np.random.choice(
            (round_5['R5WX'], round_5['R5YZ']),
            p=(
                df_matrix.loc[round_5['R5WX'], round_5['R5YZ']],
                df_matrix_opposite.loc[round_5['R5WX'], round_5['R5YZ']],
            )
        ),
    }

    return round_1 | round_2 | round_3 | round_4 | round_5 | round_6  # merge the dictionaries (python >= 3.9)

simulate_tournament()

{'R1W1': 'W01',
 'R1W8': 'W08',
 'R1W5': 'W05',
 'R1W4': 'W13',
 'R1W6': 'W06',
 'R1W3': 'W03',
 'R1W7': 'W10',
 'R1W2': 'W15',
 'R1X1': 'X01',
 'R1X8': 'X08',
 'R1X5': 'X05',
 'R1X4': 'X04',
 'R1X6': 'X06',
 'R1X3': 'X03',
 'R1X7': 'X10',
 'R1X2': 'X02',
 'R1Y1': 'Y01',
 'R1Y8': 'Y08',
 'R1Y5': 'Y05',
 'R1Y4': 'Y04',
 'R1Y6': 'Y06',
 'R1Y3': 'Y03',
 'R1Y7': 'Y10',
 'R1Y2': 'Y02',
 'R1Z1': 'Z01',
 'R1Z8': 'Z08',
 'R1Z5': 'Z12',
 'R1Z4': 'Z04',
 'R1Z6': 'Z11',
 'R1Z3': 'Z03',
 'R1Z7': 'Z10',
 'R1Z2': 'Z02',
 'R2W1': 'W01',
 'R2W4': 'W05',
 'R2W3': 'W03',
 'R2W2': 'W10',
 'R2X1': 'X01',
 'R2X4': 'X04',
 'R2X3': 'X03',
 'R2X2': 'X10',
 'R2Y1': 'Y01',
 'R2Y4': 'Y05',
 'R2Y3': 'Y06',
 'R2Y2': 'Y02',
 'R2Z1': 'Z01',
 'R2Z4': 'Z04',
 'R2Z3': 'Z03',
 'R2Z2': 'Z02',
 'R3W1': 'W01',
 'R3W2': 'W10',
 'R3X1': 'X01',
 'R3X2': 'X10',
 'R3Y1': 'Y01',
 'R3Y2': 'Y06',
 'R3Z1': 'Z01',
 'R3Z2': 'Z02',
 'R4W1': 'W01',
 'R4X1': 'X01',
 'R4Y1': 'Y01',
 'R4Z1': 'Z01',
 'R5WX': 'W01',
 'R5YZ': 'Y01',
 'R6CH':

In [7]:
from tqdm.autonotebook import trange

np.random.seed(22)

results = tuple(
    simulate_tournament()
    for _ in trange(30_000)
)

len(results)

C:\Users\mhugh\AppData\Local\Temp\ipykernel_20076\402420954.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


  0%|          | 0/30000 [00:00<?, ?it/s]

30000

In [8]:
np.random.seed(23)

my_candidates = tuple(
    simulate_tournament()
    for _ in trange(20_000)
)

len(my_candidates)

  0%|          | 0/20000 [00:00<?, ?it/s]

20000

In [9]:
def bracket_to_team_placements(bracket: dict) -> dict:
    """
    Convert bracket to dict of how many games each team won
    """
    round_1 = {
        bracket[f'R1{region}{i}']: 1
        for region in ('W', 'X', 'Y', 'Z')
        for i in (1, 8, 5, 4, 6, 3, 7, 2)
    }

    round_2 = {
        bracket[f'R2{region}{i}']: 2
        for region in ('W', 'X', 'Y', 'Z')
        for i in (1, 4, 3, 2)
    }

    round_3 = {
        bracket[f'R3{region}{i}']: 3
        for region in ('W', 'X', 'Y', 'Z')
        for i in (1, 2)
    }

    round_4 = {
        bracket[f'R4{region}{i}']: 4
        for region in ('W', 'X', 'Y', 'Z')
        for i in (1, )
    }

    round_5 = {
        bracket['R5WX']: 5,
        bracket['R5YZ']: 5,
    }

    round_6 = {
        bracket['R6CH']: 6,
    }

    return round_1 | round_2 | round_3 | round_4 | round_5 | round_6  # merge the dictionaries (python >= 3.9)

bracket_to_team_placements(results[0])

{'W01': 4,
 'W09': 1,
 'W05': 1,
 'W04': 2,
 'W06': 3,
 'W03': 1,
 'W07': 2,
 'W02': 1,
 'X01': 6,
 'X09': 1,
 'X05': 1,
 'X04': 2,
 'X11': 1,
 'X03': 3,
 'X07': 1,
 'X02': 2,
 'Y01': 5,
 'Y09': 1,
 'Y05': 1,
 'Y04': 2,
 'Y06': 1,
 'Y03': 2,
 'Y07': 1,
 'Y02': 3,
 'Z01': 2,
 'Z09': 1,
 'Z05': 4,
 'Z04': 1,
 'Z11': 2,
 'Z14': 1,
 'Z07': 1,
 'Z02': 3}

In [10]:
from tqdm.autonotebook import tqdm

team_placements = tuple(bracket_to_team_placements(result) for result in tqdm(results))

len(team_placements)

  0%|          | 0/30000 [00:00<?, ?it/s]

30000

In [11]:
team_all_placements = {
    team: np.array([team_placements_bracket.get(team, 0) for team_placements_bracket in team_placements])
    for team in ((f'{region}{i:02d}') for region in ('W', 'X', 'Y', 'Z') for i in range(1, 17))
}

team_all_placements

{'W01': array([4, 6, 6, ..., 1, 6, 3]),
 'W02': array([1, 3, 2, ..., 2, 3, 4]),
 'W03': array([1, 1, 3, ..., 4, 2, 2]),
 'W04': array([2, 2, 1, ..., 3, 2, 2]),
 'W05': array([1, 0, 2, ..., 1, 1, 0]),
 'W06': array([3, 2, 1, ..., 1, 1, 0]),
 'W07': array([2, 1, 0, ..., 0, 1, 0]),
 'W08': array([0, 0, 0, ..., 0, 1, 0]),
 'W09': array([1, 1, 1, ..., 2, 0, 1]),
 'W10': array([0, 0, 1, ..., 1, 0, 1]),
 'W11': array([0, 0, 0, ..., 0, 0, 1]),
 'W12': array([0, 1, 0, ..., 0, 0, 1]),
 'W13': array([0, 0, 0, ..., 0, 0, 0]),
 'W14': array([0, 0, 0, ..., 0, 0, 0]),
 'W15': array([0, 0, 0, ..., 0, 0, 0]),
 'W16': array([0, 0, 0, ..., 0, 0, 0]),
 'X01': array([6, 2, 2, ..., 3, 2, 3]),
 'X02': array([2, 4, 1, ..., 1, 1, 1]),
 'X03': array([3, 1, 2, ..., 6, 4, 2]),
 'X04': array([2, 3, 3, ..., 0, 3, 1]),
 'X05': array([1, 1, 1, ..., 2, 0, 2]),
 'X06': array([0, 2, 1, ..., 1, 1, 1]),
 'X07': array([1, 1, 0, ..., 0, 2, 0]),
 'X08': array([0, 0, 1, ..., 0, 0, 0]),
 'X09': array([1, 1, 0, ..., 1, 1, 1]),


Get probabilities of surviving

In [12]:
team_model_probs = dict(
    (
        t, 
        np.array(tuple(
            (v >= round).mean()
            for round in range(1, 7)
        ))
    )
    for t, v in team_all_placements.items()
)

team_model_probs

{'W01': array([0.9906    , 0.9577    , 0.88066667, 0.80466667, 0.7004    ,
        0.61523333]),
 'W02': array([0.96833333, 0.7555    , 0.49803333, 0.085     , 0.03686667,
        0.01593333]),
 'W03': array([0.93113333, 0.70973333, 0.34583333, 0.04543333, 0.0158    ,
        0.00566667]),
 'W04': array([0.88003333, 0.66786667, 0.0849    , 0.04396667, 0.01486667,
        0.00496667]),
 'W05': array([6.93100000e-01, 2.24200000e-01, 1.51666667e-02, 5.93333333e-03,
        1.33333333e-03, 3.00000000e-04]),
 'W06': array([6.64333333e-01, 2.12533333e-01, 6.31333333e-02, 4.56666667e-03,
        1.20000000e-03, 1.00000000e-04]),
 'W07': array([5.75266667e-01, 1.61266667e-01, 5.38333333e-02, 3.73333333e-03,
        6.66666667e-04, 3.33333333e-05]),
 'W08': array([4.75866667e-01, 1.69000000e-02, 6.46666667e-03, 1.86666667e-03,
        3.66666667e-04, 3.33333333e-05]),
 'W09': array([5.24133333e-01, 2.53000000e-02, 9.00000000e-03, 2.56666667e-03,
        3.33333333e-04, 3.33333333e-05]),
 'W10':

In [13]:
region_seed_to_id = dict(zip(df_seeds['Region Seed'], df_seeds['TeamID']))

df_model = pd.DataFrame({
    'Team': team_model_probs.keys()
})

df_model[[f'Round {i}' for i in range(1, 7)]] = np.array(tuple(df_model['Team'].map(team_model_probs)))

df_model.insert(1, 'TeamID', df_model['Team'].map(region_seed_to_id))

df_model.sort_values(by=['Round 6'], ascending=False, inplace=True, ignore_index=True)

df_model.head(10)

,Team,TeamID,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6
0,W01,3376,0.990600,0.957700,0.880667,0.804667,0.700400,0.615233
1,Y01,3234,0.984767,0.834167,0.644933,0.402933,0.252767,0.072733
2,Z03,3163,0.975233,0.868533,0.642500,0.452267,0.244467,0.063933
3,X02,3390,0.977167,0.802300,0.577233,0.361400,0.097533,0.057133
4,X01,3400,0.981233,0.818733,0.611800,0.342933,0.086900,0.047267
5,Y02,3417,0.960767,0.810467,0.532033,0.293700,0.170500,0.041700
6,Z01,3425,0.968333,0.779867,0.510933,0.253400,0.109100,0.020967
7,W02,3323,0.968333,0.755500,0.498033,0.085000,0.036867,0.015933
8,Y03,3261,0.977700,0.742833,0.345867,0.156533,0.078067,0.015600
9,Z02,3326,0.971033,0.717300,0.267367,0.135433,0.054100,0.011100


In [14]:
df_teams = pd.read_csv(r'..\data\unprocessed\kaggle\WTeams.csv')

df_teams

,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M
...,...,...
371,3474,Queens NC
372,3475,Southern Indiana
373,3476,Stonehill
374,3477,TX A&M Commerce


In [15]:
id_to_team = dict(zip(df_teams['TeamID'], df_teams['TeamName']))

df_model.insert(df_model.columns.get_loc('TeamID') + 1, 'Name', df_model['TeamID'].map(id_to_team))

df_model.head(25)

,Team,TeamID,Name,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6
0,W01,3376,South Carolina,0.990600,0.957700,0.880667,0.804667,0.700400,0.615233
1,Y01,3234,Iowa,0.984767,0.834167,0.644933,0.402933,0.252767,0.072733
2,Z03,3163,Connecticut,0.975233,0.868533,0.642500,0.452267,0.244467,0.063933
3,X02,3390,Stanford,0.977167,0.802300,0.577233,0.361400,0.097533,0.057133
4,X01,3400,Texas,0.981233,0.818733,0.611800,0.342933,0.086900,0.047267
5,Y02,3417,UCLA,0.960767,0.810467,0.532033,0.293700,0.170500,0.041700
6,Z01,3425,USC,0.968333,0.779867,0.510933,0.253400,0.109100,0.020967
7,W02,3323,Notre Dame,0.968333,0.755500,0.498033,0.085000,0.036867,0.015933
8,Y03,3261,LSU,0.977700,0.742833,0.345867,0.156533,0.078067,0.015600
9,Z02,3326,Ohio St,0.971033,0.717300,0.267367,0.135433,0.054100,0.011100


Get public brackets

In [16]:
df_public = pd.read_csv(f'..\data\preprocessed\womens_yahoo\{season}_womens_yahoo_picks.csv')

df_public.sort_values(by=['Round 6'], ascending=False, ignore_index=True).head(10)

,Team,TeamID,Region Seed,Seed,Region,Play In,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6
0,South Carolina,3376,W01,1,W,False,0.9831,0.9065,0.8529,0.7587,0.6604,0.3973
1,Iowa,3234,Y01,1,Y,False,0.9798,0.9446,0.8873,0.6741,0.5607,0.3168
2,Connecticut,3163,Z03,3,Z,False,0.9551,0.8523,0.5694,0.3260,0.0954,0.0429
3,LSU,3261,Y03,3,Y,False,0.9508,0.8264,0.5578,0.1615,0.1055,0.0374
4,USC,3425,Z01,1,Z,False,0.9674,0.8767,0.7212,0.3971,0.0898,0.0303
5,Stanford,3390,X02,2,X,False,0.9600,0.8362,0.6441,0.4151,0.0755,0.0247
6,Texas,3400,X01,1,X,False,0.9715,0.9040,0.7498,0.3337,0.0699,0.0240
7,Notre Dame,3323,W02,2,W,False,0.9585,0.8683,0.6675,0.1171,0.0580,0.0188
8,Ohio St.,3326,Z02,2,Z,False,0.9560,0.7998,0.3240,0.1506,0.0402,0.0177
9,UCLA,3417,Y02,2,Y,False,0.9638,0.8594,0.3344,0.0879,0.0416,0.0169


R1X1: (1, 16)
R2X1: (1, 8, 9, 16)
R3X1: (1, 4, 5, 8, 9, 12, 13, 16)
R4X1: (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)

R1X2: (2, 15)
R2X2: (2, 7, 10, 15)
R3X2: (2, 3, 6, 7, 10, 11, 14, 15)

R1X3: (3, 14)
R2X3: (3, 6, 11, 14)

R1X4: (4, 13)
R2X4: (4, 5, 12, 13)

R1X5: (5, 12)

R1X6: (6, 11)

R1X7: (7, 10)

R1X8: (8, 9)

Partition data into possible areas in advance

In [17]:
public_dict = {
    # ROUND 6
    'R6CH': df_public.loc[:, ['Region Seed', 'Region', 'Seed', 'Round 6']].rename(columns={'Round 6': 'P'}).reset_index(drop=True),

    # ROUND 5
    'R5WX': df_public.loc[df_public['Region'].isin(('W', 'X')), ['Region Seed', 'Region', 'Seed', 'Round 5']].rename(columns={'Round 5': 'P'}).reset_index(drop=True),
    'R5YZ': df_public.loc[df_public['Region'].isin(('Y', 'Z')), ['Region Seed', 'Region', 'Seed', 'Round 5']].rename(columns={'Round 5': 'P'}).reset_index(drop=True),
}

# ROUND 4
for region in ('W', 'X', 'Y', 'Z'):
    public_dict[f'R4{region}1'] = df_public.loc[df_public['Region'] == region, ['Region Seed', 'Region', 'Seed', 'Round 4']].rename(columns={'Round 4': 'P'}).reset_index(drop=True)

# ROUND 3
for region in ('W', 'X', 'Y', 'Z'):
    public_dict[f'R3{region}1'] = df_public.loc[(df_public['Region'] == region) & (df_public['Seed'].isin((1, 4, 5, 8, 9, 12, 13, 16))), ['Region Seed', 'Region', 'Seed', 'Round 3']].rename(columns={'Round 3': 'P'}).reset_index(drop=True)
    public_dict[f'R3{region}2'] = df_public.loc[(df_public['Region'] == region) & (df_public['Seed'].isin((2, 3, 6, 7, 10, 11, 14, 15))), ['Region Seed', 'Region', 'Seed', 'Round 3']].rename(columns={'Round 3': 'P'}).reset_index(drop=True)

# ROUND 2
for region in ('W', 'X', 'Y', 'Z'):
    public_dict[f'R2{region}1'] = df_public.loc[(df_public['Region'] == region) & (df_public['Seed'].isin((1, 8, 9, 16))), ['Region Seed', 'Region', 'Seed', 'Round 2']].rename(columns={'Round 2': 'P'}).reset_index(drop=True)
    public_dict[f'R2{region}2'] = df_public.loc[(df_public['Region'] == region) & (df_public['Seed'].isin((2, 7, 10, 15))), ['Region Seed', 'Region', 'Seed', 'Round 2']].rename(columns={'Round 2': 'P'}).reset_index(drop=True)
    public_dict[f'R2{region}3'] = df_public.loc[(df_public['Region'] == region) & (df_public['Seed'].isin((3, 6, 11, 14))), ['Region Seed', 'Region', 'Seed', 'Round 2']].rename(columns={'Round 2': 'P'}).reset_index(drop=True)
    public_dict[f'R2{region}4'] = df_public.loc[(df_public['Region'] == region) & (df_public['Seed'].isin((4, 5, 12, 13))), ['Region Seed', 'Region', 'Seed', 'Round 2']].rename(columns={'Round 2': 'P'}).reset_index(drop=True)

# ROUND 1
for region in ('W', 'X', 'Y', 'Z'):
    for i in range(1, 9):
        public_dict[f'R1{region}{i}'] = df_public.loc[(df_public['Region'] == region) & (df_public['Seed'].isin((i, 17-i))), ['Region Seed', 'Region', 'Seed', 'Round 1']].rename(columns={'Round 1': 'P'}).reset_index(drop=True)

for key, value in public_dict.items():  # make sure probabilities add to 1
    # print(value['P'].sum())

    value['P'] = value['P']/value['P'].sum()
    public_dict[key] = value

    # print(value['P'].sum())
    # print()

public_dict

{'R6CH':    Region Seed Region  Seed         P
 0          Z11      Z    11  0.000500
 1          X08      X     8  0.000999
 2          Z05      Z     5  0.002198
 3          Y15      Y    15  0.000400
 4          X14      X    14  0.000600
 ..         ...    ...   ...       ...
 59         Z01      Z     1  0.030279
 60         X05      X     5  0.001499
 61         Z12      Z    12  0.000799
 62         Z04      Z     4  0.003398
 63         Y08      Y     8  0.001099
 
 [64 rows x 4 columns],
 'R5WX':    Region Seed Region  Seed         P
 0          X08      X     8  0.002301
 1          X14      X    14  0.001101
 2          X16      X    16  0.001301
 3          W14      W    14  0.000900
 4          W13      W    13  0.001101
 5          W12      W    12  0.001201
 6          X09      X     9  0.006903
 7          X04      X     4  0.017809
 8          X11      X    11  0.001401
 9          W04      W     4  0.012906
 10         X07      X     7  0.006603
 11         W15      W

In [18]:
def fill_in_team(bracket: dict, round_: int, team: str, region: str, seed: int) -> dict:
    """
    Helper for simulate_public
    Fill in previous rounds of bracket for a team that wins in a future round
    """
    for r in range(1, min(round_ + 1, 4)):
        if seed > 2**(4-r):
            seed = 2**(5-r) + 1 - seed
        bracket[f'R{r}{region}{seed}'] = team

    if round_ >= 4:
        bracket[f'R4{region}1'] = team
    if round_ >= 5:
        if region in ('W', 'X'):
            bracket[f'R5WX'] = team
        else:
            bracket[f'R5YZ'] = team
    if round_ == 6:
        bracket[f'R6CH'] = team

    return bracket


def simulate_public():
    """
    Get bracket based on probabilities of winning each round
    """
    bracket = dict()

    # ROUND 6
    for matchup in ('R6CH', ):
        if not bracket.get(matchup):
            df_matchup = public_dict[matchup]  # all the teams that could win a specific game slot
            winner = df_matchup.loc[
                np.random.choice(
                    df_matchup.index,
                    p=df_matchup['P'],
                ),
                :
            ]
            bracket = fill_in_team(bracket, 6, winner['Region Seed'], winner['Region'], winner['Seed'])

    # ROUND 5
    for matchup in ('R5WX', 'R5YZ'):
        if not bracket.get(matchup):
            df_matchup = public_dict[matchup]  # all the teams that could win a specific game slot
            winner = df_matchup.loc[
                np.random.choice(
                    df_matchup.index,
                    p=df_matchup['P'],
                ),
                :
            ]
            bracket = fill_in_team(bracket, 5, winner['Region Seed'], winner['Region'], winner['Seed'])

    # ROUND 4
    for region in ('W', 'X', 'Y', 'Z'):
        for seed in range(1, 2):
            matchup = f'R4{region}{seed}'
            if not bracket.get(matchup):
                df_matchup = public_dict[matchup]  # all the teams that could win a specific game slot
                winner = df_matchup.loc[
                    np.random.choice(
                        df_matchup.index,
                        p=df_matchup['P'],
                    ),
                    :
                ]
                bracket = fill_in_team(bracket, 4, winner['Region Seed'], winner['Region'], winner['Seed'])



    # ROUND 3
    for region in ('W', 'X', 'Y', 'Z'):
        for seed in range(1, 3):
            matchup = f'R3{region}{seed}'
            if not bracket.get(matchup):
                df_matchup = public_dict[matchup]  # all the teams that could win a specific game slot
                winner = df_matchup.loc[
                    np.random.choice(
                        df_matchup.index,
                        p=df_matchup['P'],
                    ),
                    :
                ]
                bracket = fill_in_team(bracket, 3, winner['Region Seed'], winner['Region'], winner['Seed'])

    # ROUND 2
    for region in ('W', 'X', 'Y', 'Z'):
        for seed in range(1, 5):
            matchup = f'R2{region}{seed}'
            if not bracket.get(matchup):
                df_matchup = public_dict[matchup]  # all the teams that could win a specific game slot
                winner = df_matchup.loc[
                    np.random.choice(
                        df_matchup.index,
                        p=df_matchup['P'],
                    ),
                    :
                ]
                bracket = fill_in_team(bracket, 2, winner['Region Seed'], winner['Region'], winner['Seed'])

    # ROUND 1
    for region in ('W', 'X', 'Y', 'Z'):
        for seed in range(1, 9):
            matchup = f'R1{region}{seed}'
            if not bracket.get(matchup):
                df_matchup = public_dict[matchup]  # all the teams that could win a specific game slot
                winner = df_matchup.loc[
                    np.random.choice(
                        df_matchup.index,
                        p=df_matchup['P'],
                    ),
                    :
                ]
                bracket = fill_in_team(bracket, 1, winner['Region Seed'], winner['Region'], winner['Seed'])

    return dict(sorted(bracket.items()))

simulate_public()

{'R1W1': 'W01',
 'R1W2': 'W02',
 'R1W3': 'W03',
 'R1W4': 'W04',
 'R1W5': 'W05',
 'R1W6': 'W06',
 'R1W7': 'W07',
 'R1W8': 'W08',
 'R1X1': 'X01',
 'R1X2': 'X02',
 'R1X3': 'X03',
 'R1X4': 'X04',
 'R1X5': 'X05',
 'R1X6': 'X06',
 'R1X7': 'X10',
 'R1X8': 'X09',
 'R1Y1': 'Y01',
 'R1Y2': 'Y02',
 'R1Y3': 'Y03',
 'R1Y4': 'Y04',
 'R1Y5': 'Y05',
 'R1Y6': 'Y06',
 'R1Y7': 'Y07',
 'R1Y8': 'Y08',
 'R1Z1': 'Z01',
 'R1Z2': 'Z02',
 'R1Z3': 'Z03',
 'R1Z4': 'Z04',
 'R1Z5': 'Z05',
 'R1Z6': 'Z06',
 'R1Z7': 'Z07',
 'R1Z8': 'Z09',
 'R2W1': 'W01',
 'R2W2': 'W02',
 'R2W3': 'W03',
 'R2W4': 'W05',
 'R2X1': 'X01',
 'R2X2': 'X02',
 'R2X3': 'X03',
 'R2X4': 'X05',
 'R2Y1': 'Y01',
 'R2Y2': 'Y07',
 'R2Y3': 'Y03',
 'R2Y4': 'Y04',
 'R2Z1': 'Z01',
 'R2Z2': 'Z02',
 'R2Z3': 'Z03',
 'R2Z4': 'Z04',
 'R3W1': 'W01',
 'R3W2': 'W03',
 'R3X1': 'X01',
 'R3X2': 'X02',
 'R3Y1': 'Y01',
 'R3Y2': 'Y07',
 'R3Z1': 'Z04',
 'R3Z2': 'Z03',
 'R4W1': 'W01',
 'R4X1': 'X01',
 'R4Y1': 'Y07',
 'R4Z1': 'Z03',
 'R5WX': 'W01',
 'R5YZ': 'Y07',
 'R6CH':

In [19]:
np.random.seed(24)

public_brackets = tuple(simulate_public() for _ in trange(20_000))

len(public_brackets)

  0%|          | 0/20000 [00:00<?, ?it/s]

20000

In [20]:
np.random.seed(25)

public_candidates = tuple(simulate_public() for _ in trange(20_000))

len(public_candidates)

  0%|          | 0/20000 [00:00<?, ?it/s]

20000

In [21]:
simulation_data = {
    'results': results,
    'candidates': my_candidates + public_candidates,
    'public': public_brackets,
}

len(simulation_data)

3

In [22]:
import pickle

with open(f'../data/preprocessed/womens_simulation/{season}_womens_simulation.pkl', 'wb') as f:
    pickle.dump(simulation_data, f)

'Done'

'Done'